<h1 style="text-align: center;"> Tarea Minería de Datos y Modelización Predictiva </h1>
<h1 style="text-align: right; font-size: 24px; margin-right: 10px;">Guillermo Díaz Aguado</h1>

In [28]:
import pandas as pd
import numpy as np
from collections import defaultdict
from scipy.stats import chi2_contingency
import matplotlib.pyplot as plt
import seaborn as sns

from FuncionesMineria import (graficoVcramer, mosaico_targetbinaria, boxplot_targetbinaria, 
                           hist_targetbinaria, Transf_Auto, lm, Rsq, validacion_cruzada_lm,
                           modelEffectSizes, crear_data_modelo,analizar_variables_categoricas,
                           hist_target_categorica, Vcramer)

pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None) 




## 1. Introducción al objetivo del problema y las variables implicadas.


## 2. Importación de los datos y asignación correcta de los tipos de las variables.

Leemos del excel los datos:

In [29]:
df = pd.read_excel("DatosEleccionesEspaña.xlsx", sheet_name="DatosEleccionesEspaña")
df.head(10)

,Name,CodigoProvincia,CCAA,Population,TotalCensus,AbstentionPtge,AbstencionAlta,Izda_Pct,Dcha_Pct,Otros_Pct,Izquierda,Derecha,Age_0-4_Ptge,Age_under19_Ptge,Age_19_65_pct,Age_over65_pct,WomanPopulationPtge,ForeignersPtge,SameComAutonPtge,SameComAutonDiffProvPtge,DifComAutonPtge,UnemployLess25_Ptge,Unemploy25_40_Ptge,UnemployMore40_Ptge,AgricultureUnemploymentPtge,IndustryUnemploymentPtge,ConstructionUnemploymentPtge,ServicesUnemploymentPtge,totalEmpresas,Industria,Construccion,ComercTTEHosteleria,Servicios,ActividadPpal,inmuebles,Pob2010,SUPERFICIE,Densidad,PobChange_pct,PersonasInmueble,Explotaciones
0,Abadía,10,Extremadura,336,282,20.213,0,60.444,35.555,1.778,1,0,3.869,18.155,55.059,26.786,44.048,0.89,79.762,0.298,19.345,2.381,54.762,42.857,4.762,9.524,11.905,73.810,15.0,0.0,0.0,0.0,0.0,Otro,216.0,326.0,4507.5593,MuyBaja,3.07,1.56,28
1,Abertura,10,Extremadura,429,364,25.275,0,54.779,44.118,0.368,1,0,1.632,13.055,56.643,30.302,50.117,1.63,90.909,2.797,7.226,16.216,32.432,51.352,8.108,8.108,10.811,67.568,11.0,0.0,0.0,0.0,0.0,Otro,382.0,459.0,6270.7646,MuyBaja,-6.54,1.12,67
2,Acebo,10,Extremadura,569,569,27.241,0,44.203,53.140,0.966,0,1,1.230,9.139,54.834,36.027,49.033,0.70,78.910,0.703,18.102,8.197,36.066,55.737,22.951,9.836,13.115,49.180,49.0,0.0,0.0,0.0,0.0,Otro,918.0,674.0,5702.1000,MuyBaja,-15.58,0.62,74
3,Acehúche,10,Extremadura,822,704,30.114,1,50.813,45.325,0.000,1,0,4.258,14.964,60.098,24.938,51.095,0.12,93.917,0.487,5.109,7.407,61.111,31.482,16.667,5.556,16.667,59.259,50.0,0.0,0.0,0.0,0.0,Otro,599.0,842.0,9106.4649,MuyBaja,-2.38,1.37,66
4,Aceituna,10,Extremadura,623,540,30.185,1,44.562,49.867,0.796,0,1,3.531,15.569,59.391,25.040,48.154,0.64,93.258,0.161,4.173,15.385,48.077,36.538,21.154,0.000,11.538,61.538,22.0,0.0,0.0,0.0,0.0,Otro,394.0,625.0,4007.6141,MuyBaja,-0.32,1.58,96
5,Ahigal,10,Extremadura,1421,1263,22.565,0,53.068,45.501,0.409,1,0,3.167,13.301,56.509,30.190,47.783,0.56,92.963,0.633,6.334,4.225,47.887,47.888,21.127,5.634,12.676,56.338,90.0,5.0,18.0,56.0,11.0,ComercTTEHosteleria,1023.0,1451.0,5207.3458,MuyBaja,-2.07,1.39,281
6,Alagón del Río,10,Extremadura,923,826,30.024,1,59.343,37.890,0.692,1,0,4.009,14.843,63.489,21.668,49.296,0.98,86.241,1.192,12.459,9.091,35.065,55.844,9.091,15.584,12.987,54.545,45.0,0.0,0.0,0.0,0.0,Otro,423.0,879.0,1429.1900,MuyBaja,5.01,2.18,99999
7,Albalá,10,Extremadura,730,629,27.027,0,37.037,60.567,0.218,0,1,1.644,12.055,52.055,35.890,49.863,3.56,87.397,1.233,8.904,15.686,29.412,54.902,5.882,1.961,27.451,52.941,26.0,0.0,0.0,0.0,0.0,Otro,879.0,795.0,3849.4679,?,-8.18,0.83,123
8,Alcántara,10,Extremadura,1571,1320,22.803,0,46.810,50.736,0.785,0,1,2.419,12.985,57.608,29.407,50.223,2.04,86.505,2.864,10.503,7.767,42.718,49.515,1.942,9.709,4.854,78.641,82.0,9.0,14.0,32.0,27.0,ComercTTEHosteleria,1246.0,1630.0,55199.4783,MuyBaja,-3.62,1.26,215
9,Alcollarín,10,Extremadura,256,266,24.436,0,37.811,58.209,0.498,0,1,2.344,10.547,49.219,40.234,51.953,1.95,88.281,8.594,7.422,13.158,28.947,57.895,23.684,2.632,10.526,60.526,7.0,0.0,0.0,0.0,0.0,Otro,329.0,260.0,7991.5871,MuyBaja,-1.54,0.78,53


## 3. Analisis descriptivo del conjunto de datos.

El análisis descriptivo de datos en un DataFrame es un paso clave para comprender la estructura, distribución y posibles problemas en los datos antes de aplicar modelo. Realizaremos los siguientes pasos:
1. Inspección general del DataFrame.
2. Analísis de Valores nulos y duplicados.
3. Analísis de Variables númericas.
4. Analísis de Variables categóricas.

### 3.1. Inspección general del DataFrame.
Vamos a revisar la asignación correcta de los tipos de las variables. Un paso relativamente sencillo que nos puede ayudar a detectar posibles errores futuros con tan solo un poco de esmero.

In [30]:
df.dtypes

Name                             object
CodigoProvincia                   int64
CCAA                             object
Population                        int64
TotalCensus                       int64
AbstentionPtge                  float64
AbstencionAlta                    int64
Izda_Pct                        float64
Dcha_Pct                        float64
Otros_Pct                       float64
Izquierda                         int64
Derecha                           int64
Age_0-4_Ptge                    float64
Age_under19_Ptge                float64
Age_19_65_pct                   float64
Age_over65_pct                  float64
WomanPopulationPtge             float64
ForeignersPtge                  float64
SameComAutonPtge                float64
SameComAutonDiffProvPtge        float64
DifComAutonPtge                 float64
UnemployLess25_Ptge             float64
Unemploy25_40_Ptge              float64
UnemployMore40_Ptge             float64
AgricultureUnemploymentPtge     float64


Por lo que podemos observar, las siguientes variables tienen el error de ser `float` cuando deberian ser `int` (no tiene sentido tener un tercio de empresa en un municipio):
* totalEmpresas 
* Industria
* Construccion
* ComercTTEHosteleria
* Servicios
* inmuebles
* Pob2010
* SUPERFICIE
Realmente este error no es notable, puesto que las variables de tipo `float` se comportan de manera muy similar a las variables de tipo `int`

En la siguiente celda generamos un código que nos cuenta cuantas filas tienen este error, si fuese un error puntual se verificaría para cada caso antes de convertirlo todo a `int`, pero se puede ver que pasa en todas las columnas. 

In [31]:
mal_tipificadas = ['totalEmpresas', 'Industria', 'Construccion', 'ComercTTEHosteleria', 
                   'Servicios', 'inmuebles', 'Pob2010', 'SUPERFICIE']
for col in mal_tipificadas:
    print(f"Tenemos un total de {len(df[df[col].apply(lambda x: isinstance(x, float))])} elementos que son de tipo *float* en {col}")


Tenemos un total de 8117 elementos que son de tipo *float* en totalEmpresas
Tenemos un total de 8117 elementos que son de tipo *float* en Industria
Tenemos un total de 8117 elementos que son de tipo *float* en Construccion
Tenemos un total de 8117 elementos que son de tipo *float* en ComercTTEHosteleria
Tenemos un total de 8117 elementos que son de tipo *float* en Servicios
Tenemos un total de 8117 elementos que son de tipo *float* en inmuebles
Tenemos un total de 8117 elementos que son de tipo *float* en Pob2010
Tenemos un total de 8117 elementos que son de tipo *float* en SUPERFICIE


In [32]:
# Simplemente cambiaré los tipos de estas variables
for var in mal_tipificadas:
    df[var] = df[var].astype(str)

In [46]:
# Voy a guardar el tipo de valores guardados en cada variable
numericas = df.select_dtypes(include=['int', 'int32', 'int64','float', 'float32', 'float64']).columns
categoricas = df.select_dtypes(include=['object']).columns
print(categoricas)
categoricas_fin = [x for x in categoricas if x != 'Name']


Index(['Name', 'CCAA', 'totalEmpresas', 'Industria', 'Construccion',
       'ComercTTEHosteleria', 'Servicios', 'ActividadPpal', 'inmuebles',
       'Pob2010', 'SUPERFICIE', 'Densidad'],
      dtype='object')


He generado la lista de `categoricas_fin` porque al final el nombre de los munipicios no nos dará ninguna información

Como último paso de la inspección voy a mostrar estadísticas generales de todas las variables.

In [34]:
df.describe()

,CodigoProvincia,Population,TotalCensus,AbstentionPtge,AbstencionAlta,Izda_Pct,Dcha_Pct,Otros_Pct,Izquierda,Derecha,Age_0-4_Ptge,Age_under19_Ptge,Age_19_65_pct,Age_over65_pct,WomanPopulationPtge,ForeignersPtge,SameComAutonPtge,SameComAutonDiffProvPtge,DifComAutonPtge,UnemployLess25_Ptge,Unemploy25_40_Ptge,UnemployMore40_Ptge,AgricultureUnemploymentPtge,IndustryUnemploymentPtge,ConstructionUnemploymentPtge,ServicesUnemploymentPtge,PobChange_pct,PersonasInmueble,Explotaciones
count,8117.000000,8.117000e+03,8.117000e+03,8117.000000,8117.000000,8117.000000,8117.000000,8117.000000,8117.000000,8117.000000,8117.000000,8117.000000,8117.000000,8117.000000,8117.000000,8117.000000,8117.000000,8117.000000,8117.000000,8117.000000,8117.000000,8117.000000,8117.000000,8117.000000,8117.000000,8117.000000,8110.000000,7979.000000,8117.000000
mean,26.664654,5.722345e+03,4.247864e+03,26.501647,0.311199,34.403957,48.912166,14.669311,0.222866,0.621042,3.018274,13.564087,57.370592,29.065321,47.302297,5.618322,81.633460,4.337637,10.727261,7.320241,37.001267,55.678492,8.402873,10.009590,10.838410,58.646833,-4.900726,1.295565,2447.806456
std,14.893449,4.620418e+04,3.442344e+04,7.533438,0.463012,16.484308,19.946471,25.095942,0.416194,0.485157,2.052625,6.777445,6.818637,11.767040,4.362347,7.348700,12.287323,6.394942,8.847631,9.408801,20.319055,22.087672,12.959440,12.529462,13.282677,24.261857,10.382355,0.565993,15064.545054
min,1.000000,5.000000e+00,5.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,23.459000,-18.052000,11.765000,-8.960000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-52.270000,0.110000,1.000000
25%,13.000000,1.660000e+02,1.400000e+02,21.678000,0.000000,21.891000,38.690000,0.759000,0.000000,0.000000,1.389000,8.334000,53.845000,19.827000,45.725000,1.060000,75.806000,0.676000,4.933000,0.000000,28.571000,44.171000,0.000000,0.000000,0.000000,50.000000,-10.400000,0.850000,22.000000
50%,26.000000,5.480000e+02,4.470000e+02,26.424000,0.000000,35.165000,51.579000,1.883000,0.000000,1.000000,2.975000,13.881000,58.655000,27.559000,48.485000,3.590000,84.493000,2.190000,8.269000,5.882000,39.927000,52.000000,3.497000,7.143000,8.333000,62.000000,-4.965000,1.250000,52.000000
75%,41.000000,2.427000e+03,1.843000e+03,31.471000,1.000000,46.032000,62.187000,16.568000,0.000000,1.000000,4.533000,19.055000,61.818000,36.911000,50.000000,8.180000,90.462000,5.277000,13.891000,10.467000,46.667000,64.583000,11.741000,14.286000,14.286000,72.131000,0.090000,1.730000,137.000000
max,50.000000,3.141991e+06,2.363829e+06,57.576000,1.000000,94.117000,100.000000,100.000000,1.000000,1.000000,13.245000,33.696000,100.002000,76.472000,72.683000,71.470000,127.156000,67.308000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,138.460000,3.330000,99999.000000


### 3.2. Analísis de valores nulos. 

In [35]:
def valores_erroneos(df):
    """
    Cuenta los valores erroneos de cada variable en un dataframe
    sin tener en cuenta los valores erroneos.
    Los valores erroneos son:
        NaN o valor nulo
        Caracteres en columna numerica
        "?"
    
    Inputs:
        df: El DataFrame que contiene los datos.

    Returns:
        Df con los valores erroneos de cada columna de cada columna
    """
    nan_values = pd.DataFrame()
    quest_mark = pd.DataFrame()
    neg_values = pd.DataFrame()
    val_9999 = pd.DataFrame()
    # Si en algún momento quisiera saber donde se encuentra cada error podria saberlo con estos dataframe, antes de sumar

    for col in df.columns:
        nan_values[col] = df[col].isna() + df[col].apply(lambda col: (col == "NaN"))
        quest_mark[col] = df[col].apply(lambda col: (col == "?"))
        df[col] = df[col].replace('?', np.nan)
        if col in numericas and not "PobChange_pct":
            neg_values[col] = df[col].apply(lambda col: (col < 0))
            df[col] = df[col].apply(lambda x: np.nan if x <= 0 else x)
        if col in numericas:
            val_9999[col] = df[col].apply(lambda col: (col == 99999))
            df[col] = df[col].replace(99999, np.nan)

    nan_values = nan_values.sum(axis=1)
    quest_mark = quest_mark.sum(axis=1)
    neg_values = neg_values.sum(axis=1)
    val_9999 = val_9999.sum(axis=1)

    return pd.DataFrame({"NaN": nan_values, "?": quest_mark, 
                         "Valores negativos": neg_values, "Valores de 99999": val_9999})

erroneos = valores_erroneos(df)
# No voy a mostrar todos los valores erroneos del dataframe, ya que tiene 8000 registros, 
# Me parece mas significativo el numero total de datos erroneos
erroneos.sum(axis=0)



NaN                  145.0
?                     92.0
Valores negativos      0.0
Valores de 99999     189.0
dtype: float64

In [36]:
total_nan = df.isna().sum().sum()
total_nan

426

### 3.3. Analísis de Variables númericas.

In [37]:
numericas_df = df[numericas]
numericas_unicas = numericas_df.apply(lambda col: len(col.unique()))
numericas_unicas_df = pd.DataFrame({'Columna': numericas_unicas.index, 'Distintos': numericas_unicas.values})
numericas_unicas_df[numericas_unicas_df["Distintos"]<60]

,Columna,Distintos
0,CodigoProvincia,50
4,AbstencionAlta,2
8,Izquierda,2
9,Derecha,2


Como podemos ver las númericas no podemos pasarlas a las categoricas, ya que todas tienen mas de 60 elementos distintos.

### 3.4 Analísis de las Variables categoricas.

In [38]:
categoricas_dfs = []
for categoria in categoricas_fin:
    categoricas_n = df[categoria].value_counts()
    categoricas_per = df[categoria].value_counts(normalize=True)

    temp_df = pd.DataFrame({
        "n": categoricas_n,
        "%": categoricas_per
    })

    temp_df.index = pd.MultiIndex.from_product([[categoria], temp_df.index], names=['Categoria', 'Valor'])

    categoricas_dfs.append(temp_df)

categoricas_df = pd.concat(categoricas_dfs)

categoricas_df

n         %
Categoria           Valor                              
CCAA                CastillaLeón         2248  0.276950
                    Cataluña              947  0.116669
                    CastillaMancha        919  0.113219
                    Andalucía             773  0.095232
                    Aragón                731  0.090058
                    ComValenciana         542  0.066773
                    Extremadura           387  0.047678
                    Galicia               314  0.038684
                    Navarra               272  0.033510
                    PaísVasco             251  0.030923
                    Madrid                179  0.022052
                    Rioja                 174  0.021436
                    Cantabria             102  0.012566
                    Canarias               88  0.010841
                    Asturias               78  0.009609
                    Baleares               67  0.008254
                    Murcia                 45  0.005544
totalEmpresas       0.0                  1215  0.149686
                    5.0                   226  0.027843
                    4.0                   225  0.027720
                    6.0                   193  0.023777
                    7.0                   181  0.022299
                    8.0                   153  0.018849
                    9.0                   148  0.018233
                    10.0                  134  0.016509
                    11.0                  129  0.015893
                    12.0                  110  0.013552
                    15.0                  110  0.013552
                    17.0                  109  0.013429
                    14.0                  108  0.013305
                    13.0                  104  0.012813
                    18.0                   90  0.011088
                    16.0                   89  0.010965
                    20.0                   81  0.009979
                    22.0                   71  0.008747
                    19.0                   71  0.008747
                    21.0                   65  0.008008
                    24.0                   64  0.007885
                    23.0                   63  0.007761
                    28.0                   60  0.007392
                    27.0                   60  0.007392
                    25.0                   56  0.006899
                    29.0                   54  0.006653
                    26.0                   54  0.006653
                    32.0                   52  0.006406
                    34.0                   49  0.006037
                    35.0                   46  0.005667
                    36.0                   44  0.005421
                    40.0                   43  0.005298
                    31.0                   42  0.005174
                    30.0                   41  0.005051
                    39.0                   40  0.004928
                    37.0                   40  0.004928
                    38.0                   38  0.004682
                    43.0                   37  0.004558
                    42.0                   35  0.004312
                    41.0                   35  0.004312
                    47.0                   33  0.004066
                    50.0                   32  0.003942
                    33.0                   31  0.003819
                    49.0                   31  0.003819
                    44.0                   31  0.003819
                    45.0                   29  0.003573
                    56.0                   27  0.003326
                    71.0                   27  0.003326
                    53.0                   26  0.003203
                    51.0                   26  0.003203
                    57.0                   25  0.003080
                    67.0                   25  0.003080
                    58.0                   25  0.003080
            

Una vez arreglado el dataframe vamos a guardar la variables dependientes en objetos de *pd.Series*:
* **Y_cont** -> Para la variable dependiente continua
* **Y_dico** -> Para la variable dependiente dicotomica

Para luego después eliminar todas las posibles variables dependientes y así crear el dataframe que contiene unicamente las variables independientes.

In [39]:
Y_cont = df["AbstentionPtge"]
Y_dico = df["AbstencionAlta"]


out_variables_dependientes = ["AbstencionAlta",
                              "AbstentionPtge",
                              "Izda_Pct",
                              "Dcha_Pct",
                              "Otros_Pct",
                              "Izquierda",
                              "Derecha"]

X_miss = df.drop(columns=out_variables_dependientes)

## 4. Corrección de errores detectados

## 5. Análisis de valores atípicos. Decisiones.

Las variables dicotomicas están datadas como "int64" ya que usan los valores 1 y 0. Voy a cambiarlo a True y False para una mejor visualización.

In [40]:
def valores_atipicos(df, mode="intercuart"):
    # Primero seleccionaremos los datos que sean nuemricos
    numericas = df.select_dtypes(include=['int', 'int32', 'int64','float', 'float32', 'float64'])

    if mode == "intercuart":
        atipicos = pd.DataFrame()
        for col in numericas:
            qnt = df[col].quantile([0.25, 0.75]).dropna()
            Q1 = qnt.iloc[0]
            Q3 = qnt.iloc[1]
            H = 3 * (Q3 - Q1)

            limite_inferior = Q1 - H
            limite_superior = Q3 + H
            
            # Los valores atipicos seran:
            atipicos[col] = (df[col]<limite_inferior) | (df[col]>limite_superior)

        return atipicos.sum(axis=1)

    if mode == "std des":
        atipicos = pd.DataFrame()
        for col in numericas:
            media = df[col].mean()
            desv = df[col].std()

            limite_inferior = media - 3*desv
            limite_superior = media + 3*desv

            # Los valores atipicos seran:
            atipicos[col] = (df[col]<limite_inferior) | (df[col]>limite_superior)

        return atipicos.sum(axis=1)

#atipicos = pd.DataFrame({"Rango Intercuartilico": valores_atipicos(df_entero, "intercuart"),
                         #"Desviacion estandar": valores_atipicos(df_entero, "std des")})
    

In [41]:
#df_valores_erroneos = pd.concat([atipicos, erroneos], axis=1)
#df_valores_erroneos["Total"] = df_valores_erroneos.sum(axis=1)
#print(f"Nos quedariamos en total con {(df_valores_erroneos['Total'] == 0).sum()}")
# print(df_valores_erroneos)


## 5. Analisis de valores perdidos 

### 5.1 Imputaciones

In [42]:
def ImputacionCuant(var, tipo):
    """
    Esta función realiza la imputación de valores faltantes en una variable cuantitativa.

    Datos de entrada:
    - var: Serie de datos cuantitativos con valores faltantes a imputar.
    - tipo: Tipo de imputación ('media', 'mediana' o 'aleatorio').

    Datos de salida:
    - Una nueva serie con valores faltantes imputados.
    """

    # Realiza una copia de la variable para evitar modificar la original
    vv = var.copy()

    if tipo == 'media':
        # Imputa los valores faltantes con la media de la variable
        vv[np.isnan(vv)] = round(np.nanmean(vv), 4)
    elif tipo == 'mediana':
        # Imputa los valores faltantes con la mediana de la variable
        vv[np.isnan(vv)] = round(np.nanmedian(vv), 4)
    elif tipo == 'aleatorio':
        # Imputa los valores faltantes de manera aleatoria basada en la distribución de valores existentes
        x = vv[~np.isnan(vv)]
        frec = x.value_counts(normalize=True).reset_index()
        frec.columns = ['Valor', 'Frec']
        frec = frec.sort_values(by='Valor')
        frec['FrecAcum'] = frec['Frec'].cumsum()
        random_values = np.random.uniform(min(frec['FrecAcum']), 1, np.sum(np.isnan(vv)))
        imputed_values = list(map(lambda x: list(frec['Valor'][frec['FrecAcum'] <= x])[-1], random_values))
        vv[np.isnan(vv)] = [round(x, 4) for x in imputed_values]

    return vv

In [43]:
def ImputacionCuali(var, tipo):
    """
    Esta función realiza la imputación de valores faltantes en una variable cualitativa.

    Datos de entrada:
    - var: Serie de datos cualitativos con valores faltantes a imputar.
    - tipo: Tipo de imputación ('moda' o 'aleatorio').

    Datos de salida:
    - Una nueva serie con valores faltantes imputados.
    """

    # Realiza una copia de la variable para evitar modificar la original
    vv = var.copy()

    if tipo == 'moda':
        # Imputa los valores faltantes con la moda (valor más frecuente)
        frecuencias = vv[~vv.isna()].value_counts()
        moda = frecuencias.index[np.argmax(frecuencias)]
        vv[vv.isna()] = moda
    elif tipo == 'aleatorio':
        # Imputa los valores faltantes de manera aleatoria a partir de valores no faltantes
        vv[vv.isna()] = np.random.choice(vv[~vv.isna()], size=np.sum(vv.isna()), replace=True)

    return vv

Tenemos un pequeño problemilla con "CodigoProvincia" y es que tenemos muchos valores, pero es categorica. Con lo que: ¿Como agrupamos esta categorica?, Expongo unas cuantas soluciones y al lado los posibles problemas que puedan darse al utilizar esa agrupacion.

* Categorizar por CCAA: Pero esto daría un problema claro de colinealidad perfecta, porque es la misma que la variable "CCAA"
* Categorizar por cantidad de habitantes en la provincia (Suma de "Population"): Podría darse una colinealidad con la población, pero nos puede mostrar una realcion con las provincias mas pobladas. Y aunque no nos de una **dependencia lineal** con las variables de densidad y superficie, si que existira una relacion entre estos, ya que siguen la formula: $Densidad = \frac{Poblacion}{Superficie}$

col_cod_provincia = df_final.columns.get_loc("CodigoProvincia")
col_poblacion = df_final.columns.get_loc("Pob2010")

contador_poblacion = defaultdict(int)
for i in range(len(df_final)):
    contador_poblacion[df_final.iloc[i, col_cod_provincia]] += df_final.iloc[i, col_poblacion]

df_final["CodigoProvincia"] = df_final["CodigoProvincia"].replace(contador_poblacion)

df_final.head(100)


In [44]:
#df_fin = df_final.drop(columns=["CodigoProvincia"])

## 7. Detección de las relaciones entre las variables.

### 7.1. Detección de las relaciones entre las variables input continuas.


In [45]:
def Vcramer(v, target):
    """
    Calcula el coeficiente V de Cramer entre dos variables. Si alguna de ellas es continua, la discretiza.

    Datos de entrada:
    - v: Serie de datos categóricos o cuantitativos.
    - target: Serie de datos categóricos o cuantitativos.

    Datos de salida:
    - Coeficiente V de Cramer que mide la asociación entre las dos variables.
    """

    if v.dtype == 'float64' or v.dtype == 'int64':

        # Si v es numérica, la discretiza en intervalos y rellena los valores faltantes
        p = sorted(list(set(v.quantile([0, 0.2, 0.4, 0.6, 0.8, 1.0]))))
        v = pd.cut(v, bins=p)
        v = v.fillna(v.min())

    if target.dtype == 'float64' or target.dtype == 'int64':

        # Si target es numérica, la discretiza en intervalos y rellena los valores faltantes
        p = sorted(list(set(target.quantile([0, 0.2, 0.4, 0.6, 0.8, 1.0]))))
        target = pd.cut(target, bins=p)
        target = target.fillna(target.min())

    v = v.reset_index(drop=True)
    target = target.reset_index(drop=True)

    # Calcula una tabla de contingencia entre v y target
    tabla_cruzada = pd.crosstab(v, target)

    # Calcula el chi-cuadrado y el coeficiente V de Cramer
    chi2 = chi2_contingency(tabla_cruzada)[0]
    n = tabla_cruzada.sum().sum()
    v_cramer = np.sqrt(chi2 / (n * (min(tabla_cruzada.shape) - 1)))

    return v_cramer

def graficoVcramer(matriz, target):
    """
    Genera un gráfico de barras horizontales que muestra el coeficiente V de Cramer entre cada columna de matriz y la variable target.

    Datos de entrada:
    - matriz: DataFrame con las variables a comparar.
    - target: Serie de la variable objetivo (categórica).

    Datos de salida:
    - Gráfico de barras horizontales que muestra el coeficiente V de Cramer para cada variable.
    """

    # Calcula el coeficiente V de Cramer para cada columna de matriz y target
    salidaVcramer = {x: Vcramer(matriz[x], target) for x in matriz.columns}

    # Ordena los resultados en orden descendente por el coeficiente V de Cramer
    sorted_data = dict(sorted(salidaVcramer.items(), key=lambda item: item[1], reverse=True))

    # Crea el gráfico de barras horizontales
    plt.figure(figsize=(10, 6))
    plt.barh(list(sorted_data.keys()), list(sorted_data.values()), color='skyblue')
    plt.xlabel('V de Cramer')
    plt.show()

## 8. Construccion de regresion lineal

### 8.1 Mediante los métodos de selección clásica de variables 
